<a href="http://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module05%20-%20NLP%20Applications/Project07B%20-%20Text%20Classification%20Deep%20Learning%20CNN%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BiLSTM applied to sentiment analysis



this notebook, we will train a BiLSTM to classify the reviews about movies. 
We will use the dataset provided by nltk.




We prepare the datasets to train and test our models.

In [5]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

import re

X=[] #list to save the texts
y=[] #list to save the labels

for file_id in movie_reviews.fileids(): #this traverse all movides
    category=movie_reviews.categories(file_id)
    label=category[0]
    #You could use the following instruction to add 1 or 0. Below, you can find an example where this task is performed by the LabelEncoder class.
    #label= 1 if category[0] == 'pos' else 0
    tokens=list(movie_reviews.words(file_id))

    text=' '.join(str(word) for word in tokens)
    #we clean the text
    X.append(text)
    y.append(label)

X=list(X)
print("instances:",len(X),len(y))


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
instances: 2000 2000


## Splitting the dataset

In [6]:
# Splitting into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


## Encoding the labels

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

print('Number of positive and negative reviews in the training:', sum(y_train), len(y_train)-sum(y_train))
print('Number of positive and negative reviews in the test:', sum(y_test), len(y_test)-sum(y_test))


Number of positive and negative reviews in the training: 757 743
Number of positive and negative reviews in the test: 243 257



## Vectorizing the texts

In [8]:
import tensorflow as tf
tf.random.set_seed(1)

from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

tokenizer = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train)

#we add a word index for the pad tokens
tokenizer.word_index['<PAD>'] = 0
print("Vocabulary size of the training={}".format(len(tokenizer.word_index)))
print("Number of Documents in the training={}".format(tokenizer.document_count))

#We now transform the words to indexes
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)
#print(X_train[0].split())
#print(train_sequences[0])

MAX_SEQUENCE_LENGTH = 1000 #most texts have less than 1000 tokens

# pad dataset to a maximum review length in words

train_seq_pad = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_seq_pad = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_seq_pad.shape, test_seq_pad.shape

Vocabulary size of the training=35192
Number of Documents in the training=1500


((1500, 1000), (500, 1000))

## Defining the Model


In [9]:
VOCAB_SIZE = len(tokenizer.word_index)

EPOCHS=10
BATCH_SIZE=128

#Set True if you want to use pre-trained word embeddings to initialize the inputs
#False eoc
USEWE=False
TRAINABLE=False #False if pre-trained word embeddings can be adjusted

EMBED_SIZE = 300


# Embedding matrix

Each token is represented by a word embedding from the pre-trained word embedding model.

In [10]:
if USEWE:
    import gensim.downloader as api
    modelWE = api.load("glove-wiki-gigaword-50")
    EMBED_SIZE = 50

    #modelWE = api.load("word2vec-google-news-300")
    #EMBED_SIZE = 300

    # create a weight matrix for words in training docs
    embedding_matrix = np.zeros((VOCAB_SIZE, EMBED_SIZE))
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = modelWE[word]
            embedding_matrix[i] = embedding_vector
        except:
            #if word does not exist, we do not udpate the matrix
            pass

    print('embedding matrix created')


We now define the model

In [11]:
# fix random seed for reproducibility

from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

# create the model
model = Sequential()


if USEWE:
    model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, 
                        input_length=MAX_SEQUENCE_LENGTH,
                        weights=[embedding_matrix], trainable=TRAINABLE))
else:
    model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, 
                        input_length=MAX_SEQUENCE_LENGTH))

lstm_node=32
dropout=0.4

model.add(Bidirectional(LSTM(lstm_node, return_sequences=True, recurrent_dropout=0.2)))
# Add a dropout layer after each lstm layer
model.add(Dropout(dropout))
model.add(Bidirectional(LSTM(lstm_node, recurrent_dropout=0.2)))
model.add(Dropout(dropout))
# Add the fully connected layer with 256 nurons and relu activation
model.add(Dense(256, activation='relu'))
# Compile the model using sparse_categorical_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 300)         10557600  
                                                                 
 bidirectional (Bidirectiona  (None, 1000, 64)         85248     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 1000, 64)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 256)               1

## Model Training

In [ ]:
from keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')


# Fit the model
history=model.fit(train_seq_pad, y_train, 
          validation_split=0.1,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1, callbacks=[earlyStopping]
          )

Epoch 1/20
11/11 [==============================] - 152s 13s/step - loss: 0.6934 - acc: 0.5148 - val_loss: 0.6925 - val_acc: 0.5133
Epoch 2/20
11/11 [==============================] - 138s 13s/step - loss: 0.6877 - acc: 0.5556 - val_loss: 0.6851 - val_acc: 0.6600
Epoch 3/20
 8/11 [====================>.........] - ETA: 37s - loss: 0.6380 - acc: 0.7695

KeyboardInterrupt: ignored

## Model Evaluation

In [ ]:
# Final evaluation of the model
scores = model.evaluate(test_seq_pad, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
predictions=model.predict(test_seq_pad) 
predictions.reshape(-1)
predictions =[1 if item >= 0.5 else 0 for item in predictions]
predictions[:10]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predictions))


In [ ]:
import pandas as pd
labels = ['negative', 'positive']
pd.DataFrame(confusion_matrix(y_test, predictions), index=labels, columns=labels)

## Learning Curves

In [ ]:
import matplotlib.pyplot as plt
 
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['acc']
val_accuracy = history_dict['val_acc']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'b', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'red', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'b', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'red', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()